In [1]:
from typing import Dict
import flwr as fl

from flower_helpers import (create_model, get_weights, test, 
                            load_data, load_stored_tff)
from config import  (NUM_ROUNDS, TRAIN_SIZE, VAL_PORTION, 
                    TEST_SIZE, BATCH_SIZE, LEARNING_RATE, 
                    EPOCHS, FRAC_FIT, FRAC_EVAL, MIN_FIT,
                    MIN_EVAL, MIN_AVAIL, FIT_CONFIG_FN,
                    NUM_CLIENTS, CLIENT_RESOURCES, NON_IID,
                    RAY_ARGS, NUM_CLASSES, TFF_DATA_DIR, 
                    MODEL_NAME, PRE_TRAINED, DOUBLE_TRAIN)
from client import FlowerClient

In [2]:
if NON_IID:
    # non-iid dataset from tff (train and test are already split)
    trainloaders, valloaders, testloader = load_stored_tff(TFF_DATA_DIR, 
                                                           BATCH_SIZE,
                                                           DOUBLE_TRAIN)
else:
    # iid dataset from huggingface
    trainloaders, valloaders, testloader = load_data(MODEL_NAME, TEST_SIZE, 
                                                    TRAIN_SIZE, VAL_PORTION, 
                                                    BATCH_SIZE, NUM_CLIENTS, 
                                                    NUM_CLASSES)

Found cached dataset cifar100 (C:/Users/Jean/.cache/huggingface/datasets/cifar100/cifar100/1.0.0/f365c8b725c23e8f0f8d725c3641234d9331cd2f62919d1381d1baa5b3ba3142)


  0%|          | 0/2 [00:00<?, ?it/s]

Loading cached shuffled indices for dataset at C:\Users\Jean\.cache\huggingface\datasets\cifar100\cifar100\1.0.0\f365c8b725c23e8f0f8d725c3641234d9331cd2f62919d1381d1baa5b3ba3142\cache-fa578d015bc4721c.arrow
Loading cached shuffled indices for dataset at C:\Users\Jean\.cache\huggingface\datasets\cifar100\cifar100\1.0.0\f365c8b725c23e8f0f8d725c3641234d9331cd2f62919d1381d1baa5b3ba3142\cache-ebaa3534d5b9a71d.arrow
Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [3]:
net = create_model(MODEL_NAME, NUM_CLASSES, PRE_TRAINED)
init_weights = get_weights(net)
MODEL_CONFIG = net.config
# Convert the weights (np.ndarray) to parameters (bytes)
init_param = fl.common.ndarrays_to_parameters(init_weights)
# del the net as we don't need it anymore
del net

In [4]:
# server side evaluation function
def evaluate(server_round: int, params: fl.common.NDArrays,
             config: Dict[str, fl.common.Scalar]):
    data_size, metrics = test(MODEL_CONFIG, params, testloader)
    # changing the name of the metric to avoid confusion
    metrics['test_loss'] = metrics.pop('loss')
    metrics['test_accuracy'] = metrics.pop('accuracy')
    return metrics['test_loss'], metrics

def weighted_average_eval(metrics):
    weighted_train_loss = 0
    weighted_train_accuracy = 0
    for c in metrics: # c is a tuple (num_examples, metrics) for each client
        weighted_train_loss += c[0] * c[1]['val_loss']
        weighted_train_accuracy += c[0] * c[1]['val_accuracy']
    
    aggregated_metrics = {'val_loss': weighted_train_loss / sum([c[0] for c in metrics]),
            'val_accuracy': weighted_train_accuracy / sum([c[0] for c in metrics])}
    print('\t',aggregated_metrics)
    return aggregated_metrics

def weighted_average_fit(metrics):
    # print(metrics)
    weighted_train_loss = 0
    weighted_train_accuracy = 0
    for c in metrics: # c is a tuple (num_examples, metrics) for each client
        # metrics for each epoch is included, we only need the last one
        weighted_train_loss += c[0] * c[1]['train_loss'][-1]
        weighted_train_accuracy += c[0] * c[1]['train_accuracy'][-1]
    
    aggregated_metrics = {'train_loss': weighted_train_loss / sum([c[0] for c in metrics]),
            'train_accuracy': weighted_train_accuracy / sum([c[0] for c in metrics])}
    print('\t',aggregated_metrics)
    return aggregated_metrics

In [5]:
strategy = fl.server.strategy.FedAvg(
    fraction_fit=FRAC_FIT,
    fraction_evaluate=FRAC_EVAL,
    min_fit_clients=MIN_FIT,
    min_evaluate_clients=MIN_EVAL,
    min_available_clients=MIN_AVAIL,
    
    fit_metrics_aggregation_fn=weighted_average_fit,
    evaluate_metrics_aggregation_fn=weighted_average_eval,
    evaluate_fn=evaluate,
    on_fit_config_fn=FIT_CONFIG_FN,
    
    initial_parameters=init_param,
)

In [6]:
print('num clients:', NUM_CLIENTS)
print('num rounds:', NUM_ROUNDS)
print('--'*20)
print('client training set size:', [len(t.dataset) for t in trainloaders])
print('client validation set size:', [len(v.dataset) for v in valloaders])
print('test set size:', len(testloader.dataset))
print('--'*20)
print('model name:', MODEL_NAME)
print('num classes:', NUM_CLASSES)
print('pre-trained:', PRE_TRAINED)
print('learning rate:', LEARNING_RATE)
print('batch size:', BATCH_SIZE)
print('epochs:', EPOCHS)

num clients: 5
num rounds: 50
----------------------------------------
client training set size: [180, 180, 180, 180, 180]
client validation set size: [20, 20, 20, 20, 20]
test set size: 100
----------------------------------------
model name: facebook/convnext-tiny-224
num classes: 100
pre-trained: True
learning rate: 0.0001
batch size: 32
epochs: 1


In [7]:
fl.simulation.start_simulation(
    client_fn=lambda cid: FlowerClient(MODEL_CONFIG, trainloaders[int(cid)], valloaders[int(cid)]),
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=NUM_ROUNDS),
    strategy=strategy,
    client_resources=CLIENT_RESOURCES,
    ray_init_args=RAY_ARGS,
)

INFO flwr 2023-04-14 10:29:59,076 | app.py:145 | Starting Flower simulation, config: ServerConfig(num_rounds=50, round_timeout=None)
2023-04-14 10:30:01,281	INFO worker.py:1529 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
INFO flwr 2023-04-14 10:30:02,451 | app.py:179 | Flower VCE: Ray initialized with resources: {'node:127.0.0.1': 1.0, 'object_store_memory': 7903506432.0, 'GPU': 1.0, 'CPU': 24.0, 'memory': 15807012864.0}
INFO flwr 2023-04-14 10:30:02,452 | server.py:86 | Initializing global parameters
INFO flwr 2023-04-14 10:30:02,452 | server.py:266 | Using initial parameters provided by strategy
INFO flwr 2023-04-14 10:30:02,452 | server.py:88 | Evaluating initial parameters
INFO flwr 2023-04-14 10:30:07,377 | server.py:91 | initial parameters (loss, other metrics): 18.458995819091797, {'test_loss': 18.458995819091797, 'test_accuracy': 0.0}
INFO flwr 2023-04-14 10:30:07,378 | server.py:101 | FL starting
DEBUG flwr 2023-04-14 10:30:07,379 | server.py:215 | f

	 {'train_loss': 4.612055540084839, 'train_accuracy': 0.027777777777777776}


INFO flwr 2023-04-14 10:31:47,672 | server.py:116 | fit progress: (1, 18.382328033447266, {'test_loss': 18.382328033447266, 'test_accuracy': 0.0}, 100.29325360000075)
DEBUG flwr 2023-04-14 10:31:47,672 | server.py:165 | evaluate_round 1: strategy sampled 2 clients (out of 5)
DEBUG flwr 2023-04-14 10:31:52,326 | server.py:179 | evaluate_round 1 received 2 results and 0 failures
DEBUG flwr 2023-04-14 10:31:52,327 | server.py:215 | fit_round 2: strategy sampled 2 clients (out of 5)


	 {'val_loss': 4.589823961257935, 'val_accuracy': 0.0}


DEBUG flwr 2023-04-14 10:33:24,041 | server.py:229 | fit_round 2 received 2 results and 0 failures


...

DEBUG flwr 2023-04-14 11:54:09,996 | server.py:229 | fit_round 50 received 2 results and 0 failures


	 {'train_loss': 1.9750043749809265, 'train_accuracy': 0.9833333333333334}


INFO flwr 2023-04-14 11:54:15,024 | server.py:116 | fit progress: (50, 13.34089183807373, {'test_loss': 13.34089183807373, 'test_accuracy': 0.42}, 5047.6451025999995)
DEBUG flwr 2023-04-14 11:54:15,024 | server.py:165 | evaluate_round 50: strategy sampled 2 clients (out of 5)
DEBUG flwr 2023-04-14 11:54:19,596 | server.py:179 | evaluate_round 50 received 2 results and 0 failures
INFO flwr 2023-04-14 11:54:19,596 | server.py:144 | FL finished in 5052.2177281999975
INFO flwr 2023-04-14 11:54:19,611 | app.py:202 | app_fit: losses_distributed [(1, 4.589823961257935), (2, 4.566037654876709), (3, 4.5126707553863525), (4, 4.488516330718994), (5, 4.457359313964844), (6, 4.400131702423096), (7, 4.39225697517395), (8, 4.354249954223633), (9, 4.346291542053223), (10, 4.2544896602630615), (11, 4.301051616668701), (12, 4.220438480377197), (13, 4.2056663036346436), (14, 4.199428081512451), (15, 4.197626829147339), (16, 4.070887565612793), (17, 4.0748610496521), (18, 4.093803644180298), (19, 4.042614

	 {'val_loss': 3.4604979753494263, 'val_accuracy': 0.2}


History (loss, distributed):
	round 1: 4.589823961257935
	round 2: 4.566037654876709
	round 3: 4.5126707553863525
	round 4: 4.488516330718994
	round 5: 4.457359313964844
	round 6: 4.400131702423096
	round 7: 4.39225697517395
	round 8: 4.354249954223633
	round 9: 4.346291542053223
	round 10: 4.2544896602630615
	round 11: 4.301051616668701
	round 12: 4.220438480377197
	round 13: 4.2056663036346436
	round 14: 4.199428081512451
	round 15: 4.197626829147339
	round 16: 4.070887565612793
	round 17: 4.0748610496521
	round 18: 4.093803644180298
	round 19: 4.042614817619324
	round 20: 3.921111822128296
	round 21: 3.9370113611221313
	round 22: 3.905157208442688
	round 23: 3.9852352142333984
	round 24: 3.773484945297241
	round 25: 3.9455636739730835
	round 26: 3.9497320652008057
	round 27: 3.755397915840149
	round 28: 3.868858218193054
	round 29: 3.7905853986740112
	round 30: 3.6865313053131104
	round 31: 3.6423521041870117
	round 32: 3.5697191953659058
	round 33: 3.5546778440475464
	round 34: 3.7